In [30]:
import os
import random
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass



In [22]:
@dataclass
class Command:
    vocab_file: str
    dict_file: str
    use_gpu: bool = False
    interactive: bool = False
    shmem: bool = False
    multi_color: bool = False
    seed: int = -1

def form_command(c: Command): 
    cmd = [f'./solver -d {c.dict_file} -v {c.vocab_file}']
    if c.use_gpu:
        cmd.append('-g')
    if c.interactive:
        cmd.append('-i')
    if c.shmem:
        cmd.append('-m')
    if c.multi_color:
        cmd.append('-c')
    if c.seed != -1:
        cmd.extend(['-s', f'{c.seed}'])
    return ' '.join(cmd).split(' ')

def parse_iteration_info(out: str):
    ret = {}
    o = out.split('\n')
    for line in o:
        if 'Occupancy' in line:
            ret['occ'] = float(line.split(': ')[-1])
        elif 'Expected Information' in line:
            ret['I_e'] = float(line.split(': ')[-1])
        elif 'Actual Information' in line:
            ret['I_a'] = float(line.split(': ')[-1])
        elif 'Solver guessed' in line:
            ret['guess'] = line.split(': ')[-1].strip()
        elif 'Solver Iteration Time' in line:
            ret['time'] = float(line.split(': ')[-1])
    return ret
def run_command(c: Command):
    cmd = form_command(c)
    output = subprocess.check_output(cmd).decode()
    output = output.split('\n\n')
    ret = {}
    word = output[0]
    if 'Actual word' in word:
        ret['word'] = word.split(': ')[-1].strip()

    e2e_info = output[-1].split('\n')
    for line in e2e_info:
      if 'Solved in' in line:
          ret['iterations'] = int(line.split(' ')[-1])
      elif 'End to End' in line:
          ret['e2e_time'] = float(line.split(': ')[-1])

    iteration_info = output[1:-1]
    for i in range(len(iteration_info)):
        ret[f'iteration_{i}'] = parse_iteration_info(iteration_info[i])
    return ret


In [23]:
c = Command(vocab_file='./basic_dictionary/vocab.txt',
            dict_file = './basic_dictionary/potential_words.txt',
            use_gpu=True,
            shmem=True)

info = run_command(c)
print(info)

{'word': 'avian', 'iterations': 4, 'e2e_time': 1994890493.0, 'iteration_0': {'occ': 0.00198025, 'I_e': 5.9392, 'time': 1685830984.0, 'guess': 'slate', 'I_a': 4.08933}, 'iteration_1': {'occ': 0.000135634, 'I_e': 5.45817, 'time': 637374.0, 'guess': 'manor', 'I_a': 5.08746}, 'iteration_2': {'occ': 2.71267e-05, 'I_e': 2.0, 'time': 172195.0, 'guess': 'aging', 'I_a': 2.0}, 'iteration_3': {'occ': 2.71267e-05, 'I_e': 0.0, 'time': 168531.0, 'guess': 'avian', 'I_a': 0.0}}


In [37]:
seeds = [random.randint(0, 1000) for i in range(10)]
speedups = []
s_iter = []
g_iter = []
for seed in seeds:
    seq = Command(vocab_file='./basic_dictionary/vocab.txt',
            dict_file = './basic_dictionary/potential_words.txt')
    gpu = Command(vocab_file='./basic_dictionary/vocab.txt',
            dict_file = './basic_dictionary/potential_words.txt',
            use_gpu=True)
    shmem = Command(vocab_file='./basic_dictionary/vocab.txt',
            dict_file = './basic_dictionary/potential_words.txt',
            shmem=True)
    multi_color = Command(vocab_file='./basic_dictionary/vocab.txt',
            dict_file = './basic_dictionary/potential_words.txt',
            multi_color=True)
    
    results = {}
    results['seq'] = run_command(seq)
    results['gpu'] = run_command(gpu)
    print(results['seq']['word'])
    speedups.append(results['seq']['e2e_time']/ results['gpu']['e2e_time'])
    s_iter.append(results['seq']['iterations'])
    g_iter.append(results['gpu']['iterations'])
    # results['shmem'] = run_command(shmem)
    # results['color'] = run_command(multi_color)
    # columns = ['seq', 'gpu']
    # index = np.arange(len(columns))
    # plt.bar(index, [results['seq']['e2e_time'], results['gpu']['e2e_time']])
    # plt.show()
    # plt.bar(results['gpu']['e2e_time'])

print(f'Average Speedup: {np.mean(speedups)}')
print(f'Seq Iterations: {np.mean(s_iter)}')
print(f'GPU Iterations: {np.mean(g_iter)}')

droll
abled
grout
badly
empty
shone
twirl
piano
thrum
human
Average Speedup: 10.732538614478205
Seq Iterations: 3.3
GPU Iterations: 3.2
